In [1]:
import os

os.environ["TAVILY_API_KEY"] = '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'
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = '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'
os.environ["HTTP_PROXY"] = '192.168.208.104:7890'
os.environ["HTTPS_PROXY"] = '192.168.208.104:7890'

In [2]:
from langchain_community.embeddings import HuggingFaceEmbeddings

hf = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en", model_kwargs={"device": "cpu"})

/home/zhuxy/anaconda3/envs/OSAF/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/zhuxy/anaconda3/envs/OSAF/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/zhuxy/anaconda3/envs/OSAF/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If

In [3]:
os.environ["HTTP_PROXY"] = ''
os.environ["HTTPS_PROXY"] = ''
os.environ["DASHSCOPE_API_KEY"]='sk-c1bb7b06f7434e2aa6871f22f088cd78'

In [4]:
from langchain_community.vectorstores import Chroma
# vectordb = Chroma.from_documents(
#     documents=all_contents,
#     collection_name='seL4_md',
#     embedding=hf,
#     persist_directory="../vectordb"
# )
vectordb = Chroma(
    collection_name='seL4_md',
    embedding_function=hf,
    persist_directory="../vectordb"
)
retriever = vectordb.as_retriever(search_kwargs={"k": 7})

In [5]:
### Retrieval Grader

run_local = "Yes"
local_llm = "llama3.1:70b"

from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser,StrOutputParser

# LLM

llm = ChatOllama(model=local_llm, temperature=0.5, timeout=120, num_ctx=12288)
json_llm = ChatOllama(model=local_llm, format="json", temperature=0.5, timeout=120, num_ctx=12288)

grader_prompt = PromptTemplate(
    template="""You are a grader assessing relevance of a retrieved document to a user question. \n 
    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: I want to formalize seL4, and the first step is to write specifications based on properties.\n
    Please try to provide some properties of {question}. \n
    If the document is useful for answering the question, or provide the relevent Isabelle code, grade it as useful. \n
    If the document is relevant but not useful, or not relevant at all, grade it as not useful. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is useful to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explanation.""",
    input_variables=["question", "document"],
)

retrieval_grader = grader_prompt | json_llm | JsonOutputParser()
question = "Kenel Heap Access"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[0].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))
print(doc_txt)

/home/zhuxy/anaconda3/envs/OSAF/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'score': 'no'}
The rewrite rules aim to transform terms referencing the heap state through "lift p h 0" and "liftτ g (heap-update *p v h* 0, d)" into terms only referencing the original heap state "liftτ g (h, d)". The desired result is achieved by applying rewrites that reduce these terms. It's assumed that validity information for all pointers in the terms to be reduced is available. Only simple blocks are considered, and weakest pre-condition loop proof obligations have similar appearances. Function call rules introduce multiple heap states, requiring multiple rewrites targeting different heap type descriptions.


In [6]:
# Distiller

distill_prompt = PromptTemplate(
    template="""You are an assistant specialized in information distillation. \n 
    You are given a document and a question.
    Here is the document: \n\n {document} \n\n
    Here is the question: Please try to provide some properties of {question}. \n
    Here is what you need to do:
    Please distill the document while retaining all information that is useful for answering the question.
    Please answer in concise language while ensuring that as little useful information as possible is lost.
    Provide the answer with and only with the distilled documents,no openning or closing statement.""",
    input_variables=["question", "document"],
)

distiller = distill_prompt | llm | StrOutputParser()
question = "IPC"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[1].page_content
# try:
#     print(distiller.invoke({"question": question, "document": doc_txt}))
# except Exception as e:
#     print("TimeoutError")
# print("---")
# print(doc_txt)

In [7]:
from langchain_core.output_parsers import StrOutputParser
### Question Re-writer

# LLM

# Prompt
re_write_prompt = PromptTemplate(
    template="""You a question re-writer that converts an input question to a better version that is optimized \n 
     for vectorstore retrieval. Look at the initial and formulate an improved question. \n
     Here is the initial question: \n\n I want to formalize the IPC of seL4, and the first step is to write specifications based on properties.\n
    Please try to provide some properties of {question}. Improved question with no preamble, answer with the improved question only: \n """,
    input_variables=["generation", "question"],
)

question_rewriter = re_write_prompt | llm | StrOutputParser()
# question_rewriter.invoke({"question": 'Please try to find relevent content of the following property of seL4:'+question})

In [8]:
from langchain.prompts import PromptTemplate

generation_prompt = PromptTemplate(
    input_variables=["concept",'context'],
    template="""Please help me with formalizing the seL4 OS by writing specifications based on its properties and behavior. Now, based on the provided context below, suggest properties to specify. Limit suggestions to what can be directly inferred from the context; avoid unnecessary assumptions.  
     Concept:
    {concept}
     Context:
    {context}
     Answer:"""
)

merge_prompt = '''
Merge and refine the suggested properties into concise tips.
Combine similar or related properties to eliminate duplication.
Ensure each tip maintains the same level of detail.
Respond in JSON format as follows:
{
"specifications": [
  "Name: Detail",
  "Name: Detail",
  ...
]
}
Do not include any additional text outside of this JSON structure.
'''

# Post-processing
def format_docs(docs):
    joined_docs = []
    for doc in docs:
        content = doc.metadata['content']
        paragraph = doc.metadata['paragraph']
        section = doc.metadata['section'] 
        joined_docs.append(f"Paragraph:{paragraph} - Section:{section}\n{content}")
    
    return '\n\n'.join(joined_docs)

In [9]:
# ### Generate
from langchain_community.chat_models import ChatOllama
from langchain.schema.output_parser import StrOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# memory = ConversationBufferMemory()
# conversation = ConversationChain(
#     llm=json_llm,
#     memory=memory,
# )

# concept = 'Parent:ARM-Specific Data Types.Current Concept to specify:Architecture-specific state'

# context = retriever.invoke(concept)
# context = format_docs(context)
# # Run
# response1 = conversation.predict(input=generation_prompt.format(context=context, concept=concept))


# response2 = conversation.predict(input=merge_prompt)

# print(response2)

In [10]:
from typing import List

from typing_extensions import TypedDict


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        web_search: whether to add search
        documents: list of documents
    """

    question: str
    generation: str
    web_search: str
    documents: List[str]
    better_question: str
    web_documents: List[str]

In [11]:
### Search

from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(max_results=4)

In [20]:
# 本地

from langchain.schema import Document



def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.get_relevant_documents(question)
    # doc_content = []
    # for d in documents:
    #     doc_content.append(d.page_content)
    return {"documents": documents, "question": question}


def distill(state):
    print("---DISTILL---")
    question = state["question"]
    documents = state["documents"]
    web_documents = state["web_documents"]

    if web_documents:
        for d in web_documents:
            documents.append(d)

    return {"documents": documents, "question": question}
    
    # #RAG generation
    # distill_docs = []
    # for d in documents:
    #     if len(d.page_content)<200:
    #         distill_docs.append(d.page_content)
    #     else:
    #         try_count = 0
    #         max_tries = 3  # 可以设置最大重试次数
    #         while try_count < max_tries:
    #             try:
    #                 distill_doc = distiller.invoke({"document": d, "question": question})
    #                 distill_docs.append(distill_doc)
    #                 break
    #             except Exception as e:
    #                 try_count += 1
            
    # return {"documents": distill_docs, "question": question}

def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    context = format_docs(documents)

    # RAG generation
    try_count = 0
    max_tries = 3  # 可以设置最大重试次数
    while try_count < max_tries:
        try:
            memory = ConversationBufferMemory()
            conversation = ConversationChain(
                llm=json_llm,
                memory=memory,
            )
            # Run
            conversation.predict(input=generation_prompt.format(context=context, concept=question))
            generation = conversation.predict(input=merge_prompt)
            break
        except Exception as e:
            try_count += 1
            
    return {"documents": documents, "question": question, "generation": generation}


def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    
    question = state["question"]
    if state["web_documents"]:
        documents = state["web_documents"]
    else:
        documents = state["documents"]

    # Score each doc
    filtered_docs = []
    # web_search = "No"
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d}
        )
        grade = score["score"]
        if grade == "yes":
            print("---GRADE: DOCUMENT USEFUL---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT USEFUL---")
            # web_search = "Yes"
            continue
    return {"documents": filtered_docs, "question": question}

def transform_query(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    if state['better_question']:
        question = state['better_question']
    else:
        question = state["question"]
    documents = state["documents"]

    # Re-write question
    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "better_question": better_question}


def web_search(state):
    """
    Web search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """
    return
    # print("---WEB SEARCH---")
    
    # question = state["better_question"]
    # web_documents = []

    # Web search
    # docs = web_search_tool.invoke({"query": question})
    # print(docs)
    # for d in docs:
    #     web_documents.append(d["content"])

    #return {"web_documents": web_documents}


### Edges


def decide_to_generate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    # state["question"]
    # web_search = state["web_search"]
    len_doc = len(state["documents"]) if state["documents"] else 0

    print(len_doc)
    # if web_search == "Yes":
    if len_doc<2:
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print(
            "---DECISION: NEED MORE DOC, TRANSFORM QUERY---"
        )
        return "transform_query"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: DISTILL AND GENERATE---")
        return "distill"

In [27]:
# rag

from langchain.schema import Document
import os
from openai import OpenAI
from random import randint



def get_response(messages):
    key = ['sk-a6c628b29e2f4b65b0ecd322f21e669f','sk-f4ebc969b87e41d7bf1dc91dd5026bba','sk-0fc3a02ca767436d91bea234f2f9d4e7','sk-de00e7083c4542458bfdcdc8158374a7','sk-68921001062244bd9d71139b015d1b9c']

    with open('./flag.txt','r') as file:
        i = int(file.read().strip())
    
    with open('./flag.txt','w') as file:
        file.write(str(i+1))

    client = OpenAI(
        api_key=key[i%5],
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1", 
    )
    completion = client.chat.completions.create(model="llama3.1-405b-instruct", messages=messages)
    return completion.choices[0].message.content



def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.get_relevant_documents(question)
    # doc_content = []
    # for d in documents:
    #     doc_content.append(d.page_content)
    return {"documents": documents, "question": question}


def distill(state):
    print("---DISTILL---")
    question = state["question"]
    documents = state["documents"]
    web_documents = state["web_documents"]

    if web_documents:
        for d in web_documents:
            documents.append(d)

    return {"documents": documents, "question": question}
    
    # #RAG generation
    # distill_docs = []
    # for d in documents:
    #     if len(d.page_content)<200:
    #         distill_docs.append(d.page_content)
    #     else:
    #         try_count = 0
    #         max_tries = 3  # 可以设置最大重试次数
    #         while try_count < max_tries:
    #             try:
    #                 distill_doc = distiller.invoke({"document": d, "question": question})
    #                 distill_docs.append(distill_doc)
    #                 break
    #             except Exception as e:
    #                 try_count += 1
            
    # return {"documents": distill_docs, "question": question}

def generate(state):
    """
    Generate answer using API call and complete two rounds of dialogue

    Args:
        state (dict): The current graph state
        generation_prompt (str): Prompt for generating the initial response
        merge_prompt (str): Prompt for merging the context and question

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]
    context = format_docs(documents)
    # Prepare messages for the API call
    messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": f'please generate Isabelle/HOL specification about the topic:{question}, based on the following documents: {context}.',
        }
    ]

    # try_count = 0
    # max_tries = 3  # 可以设置最大重试次数
    # while try_count < max_tries:
    #     try:
    #         # First round of dialogue
    #         first_generation = get_response(messages)
            
    #         assert first_generation
    #         print(first_generation)
    #         # Add the first generation to the messages list
    #         messages.append({"role": "assistant", "content": first_generation})
            
    #         # Second round of dialogue with merge prompt
    #         messages.append({"role": "user", "content": merge_prompt})
    #         second_generation = get_response(messages)
            
    #         break
    #     except Exception as e:
    #         try_count += 1
    generation = f'''Please generate an Isabelle/HOL specification on the topic:
{question}
Based on the following documents:
{context}'''
            
    return {"documents": documents, "question": question, "generation": generation}


def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    
    question = state["question"]
    if state["web_documents"]:
        documents = state["web_documents"]
    else:
        documents = state["documents"]

    # Score each doc
    filtered_docs = []
    # web_search = "No"
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d}
        )
        grade = score["score"]
        if grade == "yes":
            print("---GRADE: DOCUMENT USEFUL---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT USEFUL---")
            # web_search = "Yes"
            continue
    return {"documents": filtered_docs, "question": question}

def transform_query(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    if state['better_question']:
        question = state['better_question']
    else:
        question = state["question"]
    documents = state["documents"]

    # Re-write question
    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "better_question": better_question}


def web_search(state):
    """
    Web search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """
    return
    # print("---WEB SEARCH---")
    
    # question = state["better_question"]
    # web_documents = []

    # Web search
    # docs = web_search_tool.invoke({"query": question})
    # print(docs)
    # for d in docs:
    #     web_documents.append(d["content"])

    #return {"web_documents": web_documents}


### Edges


def decide_to_generate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    # state["question"]
    # web_search = state["web_search"]
    len_doc = len(state["documents"]) if state["documents"] else 0

    print(len_doc)
    # if web_search == "Yes":
    if len_doc<2:
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print(
            "---DECISION: NEED MORE DOC, TRANSFORM QUERY---"
        )
        return "transform_query"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: DISTILL AND GENERATE---")
        return "distill"

In [12]:
# no_example

from langchain.schema import Document
import os
from openai import OpenAI
from random import randint



def get_response(messages):
    key = ['sk-a6c628b29e2f4b65b0ecd322f21e669f','sk-f4ebc969b87e41d7bf1dc91dd5026bba','sk-0fc3a02ca767436d91bea234f2f9d4e7','sk-de00e7083c4542458bfdcdc8158374a7']

    with open('./flag.txt','r') as file:
        i = int(file.read().strip())
    
    with open('./flag.txt','w') as file:
        file.write(str(i+1))

    client = OpenAI(
        api_key=key[i%4],
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1", 
    )
    completion = client.chat.completions.create(model="llama3.1-405b-instruct", messages=messages)
    return completion.choices[0].message.content



def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.get_relevant_documents(question)
    # doc_content = []
    # for d in documents:
    #     doc_content.append(d.page_content)
    return {"documents": documents, "question": question}


def distill(state):
    print("---DISTILL---")
    question = state["question"]
    documents = state["documents"]
    web_documents = state["web_documents"]

    if web_documents:
        for d in web_documents:
            documents.append(d)

    return {"documents": documents, "question": question}
    
    # #RAG generation
    # distill_docs = []
    # for d in documents:
    #     if len(d.page_content)<200:
    #         distill_docs.append(d.page_content)
    #     else:
    #         try_count = 0
    #         max_tries = 3  # 可以设置最大重试次数
    #         while try_count < max_tries:
    #             try:
    #                 distill_doc = distiller.invoke({"document": d, "question": question})
    #                 distill_docs.append(distill_doc)
    #                 break
    #             except Exception as e:
    #                 try_count += 1
            
    # return {"documents": distill_docs, "question": question}

def generate(state):
    """
    Generate answer using API call and complete two rounds of dialogue

    Args:
        state (dict): The current graph state
        generation_prompt (str): Prompt for generating the initial response
        merge_prompt (str): Prompt for merging the context and question

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]
    context = format_docs(documents)
    # Prepare messages for the API call
    messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": generation_prompt.format(context=context, concept=question),
        }
    ]

    try_count = 0
    max_tries = 3  # 可以设置最大重试次数
    while try_count < max_tries:
        try:
            # First round of dialogue
            first_generation = get_response(messages)
            
            assert first_generation
            print(first_generation)
            # Add the first generation to the messages list
            messages.append({"role": "assistant", "content": first_generation})
            
            # Second round of dialogue with merge prompt
            messages.append({"role": "user", "content": merge_prompt})
            second_generation = get_response(messages)
            
            break
        except Exception as e:
            try_count += 1
            
    return {"documents": documents, "question": question, "generation": second_generation}


def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    
    question = state["question"]
    if state["web_documents"]:
        documents = state["web_documents"]
    else:
        documents = state["documents"]

    # Score each doc
    filtered_docs = []
    # web_search = "No"
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d}
        )
        grade = score["score"]
        if grade == "yes":
            print("---GRADE: DOCUMENT USEFUL---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT USEFUL---")
            # web_search = "Yes"
            continue
    return {"documents": filtered_docs, "question": question}

def transform_query(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    if state['better_question']:
        question = state['better_question']
    else:
        question = state["question"]
    documents = state["documents"]

    # Re-write question
    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "better_question": better_question}


def web_search(state):
    """
    Web search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """
    return
    # print("---WEB SEARCH---")
    
    # question = state["better_question"]
    # web_documents = []

    # Web search
    # docs = web_search_tool.invoke({"query": question})
    # print(docs)
    # for d in docs:
    #     web_documents.append(d["content"])

    #return {"web_documents": web_documents}


### Edges


def decide_to_generate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    # state["question"]
    # web_search = state["web_search"]
    len_doc = len(state["documents"]) if state["documents"] else 0

    print(len_doc)
    # if web_search == "Yes":
    if len_doc<2:
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print(
            "---DECISION: NEED MORE DOC, TRANSFORM QUERY---"
        )
        return "transform_query"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: DISTILL AND GENERATE---")
        return "distill"

In [28]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generatae
workflow.add_node("transform_query", transform_query)  # transform_query
workflow.add_node("web_search_node", web_search)  # web search
workflow.add_node("distill", distill)
workflow.add_node("grade_websearch", grade_documents)

# Build graph
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "distill": "distill",
    },
)
workflow.add_edge("transform_query", "web_search_node")
# workflow.add_edge("web_search_node", "generate")
workflow.add_edge("web_search_node", "grade_websearch")
workflow.add_edge("grade_websearch", "distill")
workflow.add_edge("distill","generate")
workflow.add_edge("generate", END)

# Compile
app = workflow.compile()

In [22]:
import os
from neo4j import GraphDatabase

# 从环境变量加载Neo4j连接信息
NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "neo4jneo4j")

# 创建Neo4j驱动
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

def write_queries_to_neo4j(query, valid_queries):
    def add_query_and_rel(tx, query, sub_query):
        tx.run("""
            MERGE (q:Query {name: $query})
            MERGE (sq:Query {name: $sub_query})
            MERGE (q)-[:SPECIFY]->(sq)
            """, parameters={"query": query, "sub_query": sub_query})
    
    with driver.session() as session:
        for sub_query in valid_queries:
            # 跳过自己指向自己
            if query == sub_query:
                continue
            session.write_transaction(add_query_and_rel, query, sub_query)

In [15]:
import os
from neo4j import GraphDatabase

# 从环境变量加载Neo4j连接信息
NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "neo4jneo4j")

# 创建Neo4j驱动
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# 定义查询语句，查找所有叶子节点（无出度）及其父节点
query = """
MATCH (parent:Query)-[:SPECIFY]->(n:Query)
WHERE NOT (n)-->()
RETURN parent.name AS parent_name, n.name AS leaf_node_name
"""

log_name = {}

# 执行查询并获取结果
def get_leaf_nodes_with_parents(tx):
    result = tx.run(query)

    queries = []
    for record in result:
        q = f"Parent: {record['parent_name']}. Current Concept to specify: {record['leaf_node_name']}"
        if record['parent_name']=='seL4':
            log_name[q]=record['leaf_node_name']
        elif record['parent_name'] in ['The Monad','Adding Exceptions','Catching and Handling Exceptions','Hoare Logic','Syntax']:
            log_name[q]='Nondeterministic State Monad with Failure'
        else:
            log_name[q]=record['parent_name']
        queries.append(q)
    
    return queries

with driver.session() as session:
    leaf_nodes_with_parents = session.read_transaction(get_leaf_nodes_with_parents)
    print("Leaf Nodes with Parents:", leaf_nodes_with_parents)

# 关闭驱动
driver.close()

Leaf Nodes with Parents: ['Parent: The Monad. Current Concept to specify: Nondeterminism', 'Parent: The Monad. Current Concept to specify: Failure', 'Parent: The Monad. Current Concept to specify: Generic functions on top of the state monad', 'Parent: The Monad. Current Concept to specify: The Monad Laws', 'Parent: Adding Exceptions. Current Concept to specify: Monad Laws for the Exception Monad', 'Parent: Syntax. Current Concept to specify: Syntax for the Nondeterministic State Monad', 'Parent: Syntax. Current Concept to specify: Syntax for the Exception Monad', 'Parent: Nondeterministic State Monad with Failure. Current Concept to specify: Library of Monadic Functions and Combinators', 'Parent: Catching and Handling Exceptions. Current Concept to specify: Loops', 'Parent: Hoare Logic. Current Concept to specify: Validity', 'Parent: Hoare Logic. Current Concept to specify: Determinism', 'Parent: Hoare Logic. Current Concept to specify: Non-Failure', 'Parent: Nondeterministic State Mon

/tmp/ipykernel_3335233/2900019317.py:39: DeprecationWarning: read_transaction has been renamed to execute_read
  leaf_nodes_with_parents = session.read_transaction(get_leaf_nodes_with_parents)


In [29]:
import time
from pprint import pprint
from langchain_community.chat_models import ChatOllama
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
import re

#queries = leaf_nodes_with_parents[1:]
queries = ['Enumerations']

# 获取当前时间戳的后几位作为文件名的一部分
timestamp_suffix = str(int(time.time()))[-6:]
prefix = 'test'
#output_file = f"./output/{prefix}_{timestamp_suffix}.txt"

def recursive_rag_call(query, depth=0, max_depth=1):   
    
    # 如果达到递归深度或score达到阈值，停止递归
    if depth >= max_depth:
        return
    
    # 调用 RAG 链
    inputs = {"question": query}
    for output in app.stream(inputs):
        for key, value in output.items():
            # Node
            pprint(f"Node '{key}':")
            # Optional: print full state at each node
            # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
        pprint("\n---\n")

    # Final generation
    result = value["generation"]
    with open(f"./baseline/rag/{query}.txt", "a") as f:
        f.write(result)

for query in queries:
    # 开始递归调用
    recursive_rag_call(query)

---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT USEFUL---
---GRADE: DOCUMENT NOT USEFUL---
---GRADE: DOCUMENT NOT USEFUL---
---GRADE: DOCUMENT NOT USEFUL---
---GRADE: DOCUMENT NOT USEFUL---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT NOT USEFUL---
---ASSESS GRADED DOCUMENTS---
1
---DECISION: NEED MORE DOC, TRANSFORM QUERY---
"Node 'grade_documents':"
'\n---\n'
---TRANSFORM QUERY---
"Node 'transform_query':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT USEFUL---
"Node 'grade_websearch':"
'\n---\n'
---DISTILL---
"Node 'distill':"
'\n---\n'
---GENERATE---
"Node 'generate':"
'\n---\n'


In [26]:
import time
from pprint import pprint
from langchain_community.chat_models import ChatOllama
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
import re

queries = leaf_nodes_with_parents[1:]

# 获取当前时间戳的后几位作为文件名的一部分
timestamp_suffix = str(int(time.time()))[-6:]
prefix = 'test'
#output_file = f"./output/{prefix}_{timestamp_suffix}.txt"

def recursive_rag_call(query, depth=0, max_depth=1):   
    
    # 如果达到递归深度或score达到阈值，停止递归
    if depth >= max_depth:
        return
    
    # 调用 RAG 链
    inputs = {"question": query}
    for output in app.stream(inputs):
        for key, value in output.items():
            # Node
            pprint(f"Node '{key}':")
            # Optional: print full state at each node
            # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
        pprint("\n---\n")

    # Final generation
    result = value["generation"]
    #pprint(result)
    
    
    # 获取新的子 query（假设 result 中包含子 query 列表）
    new_queries = extract_sub_queries(result)  # 提取子 query

    if len(new_queries)<=1:
        return
    
    valid_queries = []
    # 对每个新的子 query 进行递归调用
    for new_query in new_queries:
        if new_query:
            valid_queries.append(new_query)
        # 写入output文件
    with open(f"./baseline/llama70b/{log_name[query]}.txt", "a") as f:
        f.write(f"Depth: {depth}, Query:\n{query}\n")
        f.write(f"Result:\n")        
        for valid_query in valid_queries:
            f.write(f"{valid_query}\n")


    # 正则表达式匹配模式
    pattern = r"Current Concept to specify: (?P<concept>.+)$"

    # 使用正则表达式提取 leaf 部分
    match = re.search(pattern, query)

    if match:
        query = match.group("concept")
        
    #write_queries_to_neo4j(query, valid_queries)

    for valid_query in valid_queries:
        recursive_rag_call(valid_query, depth + 1, max_depth)

def extract_sub_queries(result):
    try:
        # 尝试将 JSON 字符串解析为字典
        import json
        data = json.loads(result)
        # 返回字典中的 specifications 列表
        return data.get("specifications", [])
    except json.JSONDecodeError:
        # 如果解析失败，则返回一个空列表或处理错误
        return []

for query in queries:
    # 开始递归调用
    recursive_rag_call(query)

---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT USEFUL---
---ASSESS GRADED DOCUMENTS---
7
---DECISION: DISTILL AND GENERATE---
"Node 'grade_documents':"
'\n---\n'
---DISTILL---
"Node 'distill':"
'\n---\n'
---GENERATE---
"Node 'generate':"
'\n---\n'
---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT NOT USEFUL---
---GRADE: DOCUMENT USEFUL---
---GRADE: DOCUMENT USEFUL---
---ASSESS GRADED DOCUMENTS---
6
---DECISION: DISTILL AND GENERATE---
"Node 'grade_documents':"
'\n---\n'
---DISTILL---
"Node 'distill':"
'\n---\n'
---GENERATE---
"Node 'generate':"
'\n---\n'
---RETRIEVE---
"Node 'retrieve':"
